# Качаем комментарии с youtube 

* https://medium.com/greyatom/youtube-data-in-python-6147160c5833
* https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=hopeful-vim-228218

In [1]:
import requests
import pickle
from tqdm import tqdm_notebook
from joblib import Parallel, delayed

In [2]:
with open('api_key.txt') as f:
    api_key = f.read()

В коде страницы искать id по ссылке как: "externalId":

In [3]:
base_search_url = 'https://www.googleapis.com/youtube/v3/search?'

url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, "UCs6h0i7chZZX3fWD4IOh1kQ")

req = requests.get(url)
req

<Response [403]>

In [3]:
def get_all_video_in_channel(channel_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с канала по его id
        channel_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'
    
    first_url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['id']['kind'] == "youtube#video":
                video_links.update({i['id']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_all_video_in_playlist(playlist_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с плейлиста по его id
        playlist_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/playlistItems?'
    
    first_url = base_search_url+'key={}&playlistId={}&part=id,snippet&order=date&maxResults=25'.format(api_key, playlist_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['snippet']['resourceId']['kind'] == "youtube#video":
                video_links.update({i['snippet']['resourceId']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_comment_threads(video_id, api_key, style, performer, video_dict, limit=2000):
    """
    Достёт из-под видео 500 комментов
        video_id : string 
            id видяшки
        api_key : string
            ключ доступа к API 
        style : string
            стиль музыки 
        performer : string
            исполнитель
        video_divt : dict
            словарь с соответствием id и названий видео
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/commentThreads?'
    first_url = base_search_url+'key={}&videoId={}&part=snippet&maxResults=100&textFormat=plainText'.format(api_key, video_id)

    com_inf = [ ]
    url = first_url

    while len(com_inf) < limit:
        req = requests.get(url)
        resp = req.json()

        for item in resp['items']:
            comment = item["snippet"]["topLevelComment"]
            if comment['kind'] == 'youtube#comment':
                cur_infa = {
                    'music_style' : style,
                    'performer' : performer,
                    'video_name' : video_dict[video_id],
                    'comment_id' : comment["id"],
                    'author' : comment["snippet"]["authorDisplayName"],
                    'text' : comment["snippet"]["textDisplay"],
                    'date' : comment["snippet"]['publishedAt'],
                    'likes' : comment["snippet"]['likeCount']
                }

                com_inf.append(cur_infa)       
        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break    
    return com_inf

In [ ]:
style = 'popsa'

target = {
    "loboda" : {'канал' : "UCiU8w7NabK3xFgjxmHovjCg", 'листы' : ["PL2944ACC2EC785BD5"]},
    'билан' : {'канал' : "UCrnxeDn1Biodm9jO8yAyj3g", 'листы' : ["PL57Ql2OzGW1BDEuMJ3qi6fbzYON26fUTU"]},
    'темникова' : {'канал' : "UC4EiMlCetZkEOfFK8pfRAvw", 'листы' : ["PLQ7t2Z77CsoW6oeoaMxa665eO7hWA4s-t"]},
    'serebro' : {'канал' : "UCE3faOl-pvohsu7hof1Fnsw", 'листы' : ["PLcH8mHDiTxZ0DadRGyfEpSZFA6RZ203Ln"]},
    'гагарина' : {'канал' : "UCs6h0i7chZZX3fWD4IOh1kQ", 'листы' : ["PLjtCcgIBP_Ttn3N5JvdhQRBQGqrNTaQwK"]},
    'ёлка' : {'канал' : "UC3-HSETEZNYumQ4GXUiQAuw", 'листы' : [ ]},
    'виа_гра' : {'канал' : "UCgpnEoslcNaqYuf_78avHXw", 'листы' : ["PLmR3vfNp_YiO_fNanIM1ym-l8qqRbY0-i"]},
    'бузова' : {'канал' : "UCzTom8k7UduI9jcs6umWD9Q", 'листы' : ["PLqM15F9IQUAFbQW1XjBY3c7trFyOZ9Uol"]},
    'караулова' : {'канал' : "UCDGjJgCgyrbMfyjBzuo1xiA", 'листы' : ["PLP2A5cnvW__Ur_S0EOLCoLRyOZzUdCaFm"]},
    'кока' : {'канал' : "UCbj1rZy7-qktxZTawl1S_nA", 'листы' : ["PLdnFu7fbWUIMRoI5lFpait4mxL1JzU7Go"]},
    'нюша' : {'канал' : "UCm9VWKAFz0aXpuEHPHMae7w", 'листы' : ["PLJIRUPzMCrKKvenaGgQmjCDo49o7YByyT"]},
    'макsим' : {'канал' : "UCUy_4pyOCyQK7VM6pR5WxSw", 'листы' : ["PL6E039C93D2424F75"]},
    'градусы' : {'канал' : "UCBKvd5NasOgVXB1JmCL1urg", 'листы' : ["PLHb8DB_PqBc9fypbfTj0Hh41DP3CtwDnZ"]},
    'лазарев' : {'канал' : "UCRSZzzNedo9M2629ZTKBANA", 'листы' : ["PLVwU9Hicy4XVXQ6uHamkBfDygMbN53Tx7"]},
    'барских' : {'канал' : "UCSZP_I8E-_pRFVm90S0Zy6Q", 'листы' : ["PLyqQ1XMQ458rhd1QaXWhkX3YLuEO9CRTO"]},
    'крид' : {'канал' : "UCZeinbsBPa37qf4HPj_w-lw", 'листы' : ["PLomtNzMp4OgW3gmEb-IO35ojDS83gRgAe"]},
    'тимати' : {'канал' : "UCJjMGnyycI7f4Vl_UMuDB1Q", 'листы' : ["PLFBB9F68AF5C4F818"]},
    'руки_вверх' : {'канал' : "UCf5XmQY79pqR3i2oliGqMQg", 'листы' : ["PLLf5zOiu21LGS51xGPeHeDgO8H8QIPtbs"]},
    'astudio' : {'канал' : "UC0cst8k3zK9A8BRwZ4tE3AA", 'листы' : ["PLA9467A4A29649490"]},
    'iowa' : {'канал' : "UCj2K424HxCZTLWuw-PLBGNA", 'листы' : ["PLy3FBKe3LReDB5zQMMU9F9TB2_XjlQW3n"]},
    '2маши' : {'канал' : "UCt2p-g_IT0F_yuCfz79yKRw", 'листы' : ["PLrG9pIP1c-m9fyzfQfvFMVXHedc8LO-S5"]}
    }

In [ ]:
style = 'estrada'

target = {
    'киркоров' : {'канал' : "UCJgUXBCxgiWUivJV2RmLD6g", 'листы' : ["PL1T7-Rtn2AzkT55xxmXmC01gKGXb0Br-b"]},
    'лепс' : {'канал' : "UCWopWNCej06Z3S4IG39yHLA", 'листы' : ["PLfBFsBXvPtQ5UCjScTwQxR2i00rKXN9ZM", "PLQuC5fFp9MxdUV0tNpkYcQMskJFhoFlN0"]},
    'пугачёва' : {'канал' : "UCtLuREHRfDQbKAzlomUvgzQ", 'листы' : [ ]},
    'михайлов': {'канал' : "UCc_rpSa6PXPow6aVRhlnQ8g", 'листы' : ["PLA360E570CE8C6566"]},
    'меладзе': {'канал' : "UCQKQrhLrtENPdxCNjeYUReg", 'листы' : [ ]},
    'ваенга': {'канал' : "UCyAcXWJxwjraTlzckEbgMfQ", 'листы' : ["PLS6Hv_KxWNTxZtQ3PyyfZNZaxW7D02rL8"]},
    'валерия': {'канал' : "UC8ctItMhn_FNS1c301_Q-zA", 'листы' : ["PLapk1eIugjg4wTQYSsAPNz9ZQnaH8ijSP"]},
    'агутин': {'канал' : "UCVafaAr4oDm-DurxwclK3Zg", 'листы' : [ ]},
    'басков': {'канал' : "UCJqsNPrsH71EnsP8kbtrHoA", 'листы' : ["PLU3z5rOtm15b41T47fJiX1GRqDA1LVbcP"]},
    'ротару': {'канал' : "UC0Ux11GJjRsXA-DbU6zUMkw", 'листы' : [ ]},
    'лолита' : {'канал' : "UCaKeasZJuQ_ZYw0o9hsfGbg", 'листы' : ["PL7Mdigi3VDUd2hkYvisfOEojlzkO3XZp4"]},
    'газманов': {'канал' : "UCxPUxaXn8sVLABf7HxxMabQ", 'листы' : ["PL7yxEFBiOVQYSFwc9Rn_G1tHM2epv5IxH"]},
    'николаев': {'канал' : "UC9nYweZwDnAr-kIkADlJA6A", 'листы' : ["PL23vzKgeCvbwQVzrR23jSSV5G9D_KHOOX"]},
    'леонтьев': {'канал' : "UCi0x0GBJ7095DM4qjTIwp9w", 'листы' : [ ]},
    'буйнов': {'канал' : "UCRlcuihl8_Qp657AYTC_KUA", 'листы' : [ ]}      
    }


In [ ]:
style = 'rock'

target = {
    'би_2' : {'канал' : "UC8yXiBwG4BbMtKpWuV8TwKw", 'листы' : ["PLB4966DCD2AA75A41"]},
    'ленинград' : {'канал' : "UCY0C6A3t3RTUN3BB65rWAgQ", 'листы' : ["PLkndHGjlDk4nG35TQhUrZobCTZWgI07Gm"]},
    'кино' : {'канал' : "UC8mndg3Tr2g9iMoJMI4ob5A", 'листы' : ["PL-6nI_KhZ10Rw_TnRxKFpIuCM4vxW41VX"]},
    'сплин' : {'канал' : "UCqrFVh8KBoivpwJZEKprJLA", 'листы' : ["PLsaQXb_TMEB62QPKCgNgg-ddOnoAaSHEG"]},
    'мумий_тролль' : {'канал' : "UCq4N3IijMRNvsbp9aeV6JZg", 'листы' : ["PL24C5808E49A41C43"]},
    'звери' : {'канал' : "UCBU_tBl_50vukdRbI5g5wUQ", 'листы' : ["PLlam7yXvvlZOMmoK6xUEmoLK2rMWdFS3X"]},
    'гребенщиков' : {'канал' : "UCawVzkkGUvxTvwn_c_WJ-lA", 'листы' : ["PLWuSxNGNHHaDtSdIDKze7gNYiidPQyF6I"]},
    'земфира' : {'канал' : "UCaYQXoII8_plzI_nDrmlZJQ", 'листы' : ["PLueeXBHkZzqHFWsb2J3-EDT9cEfyCWnJX"]},
    'ддт' : {'канал' : "UCWgb__tbz50Re3ocakN6_Vw", 'листы' : [ ]},
    'кукрыниксы' : {'канал' : "UCFq49ivT4GaT0FPGvj4WJ3Q", 'листы' : ["PLvU3IxYn61-4t4vhRI_T2rcvNW80ocDnB"]},
    'арбенина' : {'канал' : "UC-SeYae4GU47pQf-0lb72_Q", 'листы' : ["PL6TM8DkKQQM1q9xUS6zZoVtm10aPocrAw"]},
    'чичерина' : {'канал' : "UCENXKVDUNk6GaNzIFF0Xf5A", 'листы' : ["PLRbEiKmVKW66UTA2OBfL7lnWqADh6BhWc"]},
    'несчастный_случай' : {'канал' : "UCk8xIo1TbXPSFMemR3b19lQ", 'листы' : ["PLKbT03_kKOYTnX0ZXtgbaOV_SK0Dqibrg"]},
    'смысловые_галлюцинации' : {'канал' : "UCURsyF1culXEGs7w24xOmzQ", 'листы' : ["PLCOdOdLTv07EECaCLscazIpidglLXmsdT"]},
    'ногу_свело' : {'канал' : "UC9Lkz6OveCnO9VEFe6Wv3pQ", 'листы' : ["PLA9D5A04D6885F7E8"]},
    'агата_кристи' : {'канал' : "UCRJMtae35RCaS_GTfaxmf3g", 'листы' : []},
}


In [ ]:
style = 'metal'

target = {
    'ария' : {'канал' : "UC9GpCtyxsW6zTm2u_7LTt9Q", 'листы' : ["PLwbk0klXps3ufUSpwkv9NnixgkHdgA_I8"]},
    'louna' : {'канал' : "UCRnu_KhjPHhyOiXCLF_WlLg", 'листы' : ["PLLqM6m0hE0dijJsmL5eRJJCBGIItPplNw"]}, 
    'чёрный обелиск' : {'канал' : "UCCmJXd1oU-1pH9QNDdHM9Rg", 'листы' : [ ]} ,
    'кипелов' : {'канал' : "UCCDEukPgZgZIsGMh8nCf-jA", 'листы' : ["PLDFA9D335158220C6"]} ,
    'коррозия_метала' : {'канал' : "UCL5fqnGw2N6ubL3Mc_oHK2g", 'листы' : ["PLRE9L4YJiL01qZqvnJkfg-c-3_YKjDEnl"]}, 
    'маврин' : {'канал' : "UCdp92sFWeIl193wO2SKd6LQ", 'листы' : [ ]} ,
    'эпидемия' : {'канал' : "UCJoDCkzPaQjbKUz9zP0ho3Q", 'листы' : [ ]} ,
    'птвп' : {'канал' : "UCi7_-zvQY8dROvUDVjK7SLQ", 'листы' : [ ]} ,
    'marschak' : {'канал' : "UCs9gBVBhrKprj0eZu_iMXNQ", 'листы' : [ ]} ,
    'tracktorbowling' : {'канал' : "UCLXU3lmGchA2W4drZa4Gc8w", 'листы' : ["PL96BaF94qlvvKKHWGYPv7719HaC5FxpcV"]}  ,
    'my_autumn' : {'канал' : "UCoRVq9mSNfs8fXLVca_yhFg", 'листы' : [ ]} ,
    'molecul' : {'канал' : "UCIYYzMlRYtGy1bTkKJinC9Q", 'листы' : [ ]} ,
    'fail_emotions' : {'канал' : "UCiIse2puwr0KMc1CSmO4rgQ", 'листы' : ["PLwr99ZgDQY8zYZ328Hg6OdX9PVtj4yxWA"]} ,
    'neonate' : {'канал' : "UC--lDSnzOE_rug9YLhmpH8g", 'листы' : [ ]} ,
    'состояние_птиц' : {'канал' : "UCF1oACOoJDqX5wf-nRf-rDw", 'листы' : ["PLoVeVlPN3-FnBy0XVYxPcT1ik8TdZe069"]} ,
    'stigmata' : {'канал' : "UCbg2nHQ5cl8f5UBCEdPd9NQ", 'листы' : ["PLMb8PgOlpig83Nh_jX1xmgnq8_101MNiC"]} ,
    'idea_fix' : {'канал' : "UCMLLwpK3GFUa4cmSEY1wqiA", 'листы' : [ ]} ,
    'ena' : {'канал' : "UC9nD8XG34ElR9tiQPKRwpgg", 'листы' : [ ]} ,
    'korea' : {'канал' : "UCfAKWhsYaccd41ZjMveoSrw", 'листы' : [ ]} ,
    'gknr' : {'канал' : "UC1PtnT9u7wyX_4RpODFfqzg", 'листы' : [ ]}    ,
    'pictures_inside_me' : {'канал' : "UCublkZsf9y6CbJm6P0iErxg", 'листы' : [ ]}
 }

In [ ]:
style = 'indi'

target = {
    'монеточка' : {'канал' : "UC53JOxGaJlh6vD0ZHUjWhsA", 'листы' : ["PLgAT5eZui0q6GnAXkkqd4TumZavXKxyvi", "PLgAT5eZui0q6VgmNvNk8nwOm-2-tHC2Tv"]},
    'гречка' : {'канал' : "UCMBvRL5TIshLV2iDCRJgKCQ", 'листы' : ["PLZCo8btGoaYc0y2pGWeQjWdivsJK4JOA3"]},
    'хадн_дадн' : {'канал' : "UCpWFDabgSGgvsuLkOmDw29w", 'листы' : ["V75mV2iIeLs&list=PLgSBd00wJXIcHeOdi047PYU975ocaAAP8"]},
    'алоэ_вера' : {'канал' : "UCXger0X0bKd0adzG3dBdVhg", 'листы' : ["PL-HtAWBg31M95ez7YOV6YYZ2oDVG_6fTm"]},
    'наадя' : {'канал' : "UCuzptdNvIypH-_umvjV4xXw", 'листы' : [ ]},
    'пошлая_молли' : {'канал' : "UCDVnp5x53g5L-kBvQsGOW9w", 'листы' : ["PLrLCBBnpufnL56likFh7g_ryK3tNJHIJv"]},
    'женя_любич' : {'канал' : "UC4y0Vb3hQUTsw_sZ9nE1_uw", 'листы' : ["PLgf2biZ_ixoMyBf4ik27vwq-wzd4gYTMf"]},
    'ic3peak' : {'канал' : "UC8FTRaCE2WlR3ZJcvLSJR8g", 'листы' : [ ]},
    'rsac' : {'канал' : "UCpfnHNpQEfJY94ulWbWzUFg", 'листы' : [ ]},
    'cбпч' : {'канал' : "UCg2VrielzgQTA1fGdsRJnRg", 'листы' : [ ]},
    'буерак' : {'канал' : "UC9Qb-JPdRwx8k2cfa7v6rgQ", 'листы' : [ ]},
    'пасош' : {'канал' : "UCzlqVPjqRr6zBm4Y979pa5w", 'листы' : [ ]},
    'казускома' : {'канал' : "UCBtJCD9JbhzzeqbeQFjOw7g", 'листы' : [ ]},
    'sonic_death' : {'канал' : "UCSedMCgu0jlrUK3ifz_bsgA", 'листы' : [ ]},
    'деревянные киты' : {'канал' : "UCFOz1uLnPd-9cNMTUEe3eLw", 'листы' : [ ]},
    'электрофорез' : {'канал' : "UCkK2swbqf9ENq13OW4Ju_VQ", 'листы' : [ ]},
    'pompeya' : {'канал' : "UC1pzrD-FkbgCc8MtSBAdjRw", 'листы' : ["PL_P8WTjOs4NcX0sQi44erQdRGBUyqqBKK"]},
    'motorama' : {'канал' : "UC9Vtn5WRFoHdkb5fbXswmWw", 'листы' : []},
    'антоха_мс' : {'канал' : "UCKLObxxmmAN4bBXdRtdqEJA", 'листы' : []},
    'retuses' : {'канал' : "UCJmbZhP4x3pGJDYjpyJU7Wg", 'листы' : [ ]},
    'мальбэк' : {'канал' : "UCnE0DWdkLRoP2BcJavvDqDQ", 'листы' : ["PL_QO9QMbbcKLhxCd5OZ0n5Y7CYB7NcGz6"]},
    'гш' : {'канал' : "UCbwgX0rn7LQrVT98grYIpDg", 'листы' : [ ]},
    'краснознаменная_дивизия_имени_моей_бабушки' : {'канал' : "UCq8DFwAD58-V9sXJD5CCmuQ", 'листы' : ["PLxfB-2J9-pLnGQBYxm6YPZA6mANkqJmYB"]},
    'рыцарные_рыцари' : {'канал' : "UCm1EgHvLr1DzDxWEBRMt3VQ", 'листы' : [ ]},
    'интурист' : {'канал' : "MrJenyaGorbunov", 'листы' : [ ]}
}

In [5]:
comments_dict = { }

In [100]:
style = 'pank'

target = {
    'autoscan' : {'канал' : "UCuxp1QzyahPlNORjEmEe9iw", 'листы' : ["PLL3nbPiIl0nMRcG_NJX52VBMM4VJQC99f"]},
    'ёптыть' : {'канал' : "UCwTDWaq6s7qrHAiBITxoF6w", 'листы' : [ ]},
    'йорш' : {'канал' : "UCpFl6cp4SeUeSOpUp5o01OA", 'листы' : ["PLW0gZnAirx6TprcJMnmHfS6HZeR2nxoyn", "PLW0gZnAirx6T9BLqbFCACswKjvzsTRyr-"]},
    'fpg' : {'канал' : "UC7XEDFtS1uTDVS0u6j-MnWQ", 'листы' : [ ]},
    'элизиум' : {'канал' : "UCS5VpC1BQXWZSGnGnBFti8w", 'листы' : ["PLKP0pmm7wiTfAZXFS3Q0mtUaW9WWLF4fv"]},
    'пурген' : {'канал' : "UCQfnFmEOudWDz1ef9zgw2hQ", 'листы' : [ ]},
    'deadkedы' : {'канал' : "UCklL_clJ2Y0YiEDUY6UXkmw", 'листы' : [ ]},
    '2nd_season' : {'канал' : "UCDyBj41wllLeEf9KSDfX11A", 'листы' : [ ]},
    'кожаный_олень' : {'канал' : "UCKsQl6G1S645wu0GnaGFRBQ", 'листы' : [ ]},
    'химера' : {'канал' : "UCx7PJGn1h2_-KlC286HsDbg", 'листы' : [ ]},
    'тени_свободы' : {'канал' : "UCauSvxqQ2tlUJKoyLbrVg8w", 'листы' : [ ]},
   'княzz' : {'канал' : "UCPMZ_PFf_z3xpcJh-2S-mmQ", 'листы' : ['PL9G-MXq1lSOSuZAPpn0RGsq9APM8E7tPq']},
    'план ломоносова' : {'канал' : "UCIFOybBK_5tn_PV9PAl5VKQ", 'листы' : ["PLjxRw_O_H2EsX1CLNc1p5yMBgJeRnpR8N"]},
    'порнофильмы' : {'канал' : "UCffMEYFHnAY_qnIYJr3n-Wg", 'листы' : ["PLn0JfJEnm6fQi6rgqJ7_-x4EZTFV24cgc"]},
    'король_и_шут' : {'канал' : "UCKSClb5P4hMTmNeynTzEm7A", 'листы' : ["PLO25aJdStWmuGeEelOgIrCKAtOGQifyNS"]},
    'сектор_газа' : {'канал' : "UCnTUe--YzQw8GYwmT_3LMww", 'листы' : [ ]},
    'тараканы' : {'канал' : "UCHWQBE_V-TQLpZv-Tu2wFwQ", 'листы' : ["PL0407F996BA5AD989"]},
    'наив' : {'канал' : "UCruJRxZMwKLhnxXmRRRehAQ", 'листы' : ["PL97XakTtJGlG2yFIA0dY-JToYj37SmRBF"]},
    'гражданская_оборона' : {'канал' : "UCeMsJJOE6avjyvbqP4Kf24g", 'листы' : [ ]},
    'летов' : {'канал' : "UCOH7H-J4BAi-lR0c9MGwi7g", 'листы' : [ ]}
}

In [24]:
style = 'shanson'

target = {
     'михаил_круг' : {'канал': 'UCNCnMXQTm9lCunw-UIMFnhw', 'листы': [
        'PLI5DBAR-knxHnZlqTci8Cfi2RibpYnJpx', 
        'PLI5DBAR-knxEzq5lmZN4nrbPyXLNvoZpW', 
        'PLI5DBAR-knxELibvMI0xEJpaPb3yvnVLk'
    ]},
    'ирина_круг' : {'канал': 'UCArZO6LevX_-43iy_dqmlZg', 'листы': [
        'PLk2zkNwKIjUI2kST3Db31pHZSbe9IuANT'
    ]},
    'бутырка' : {'канал' : 'UCtZQJSi3v6O3eU_pych6ZQQ', 'листы': [
        'PLvZofFI2m4GJ-qygAfh3TofUGR9GP6NiM'
    ]},
    'трофимов' : {'канал' : 'UCk7l9hpgLbpfHJR3Ssz6Njw', 'листы': [
        'PLhjB_FUFY4p0FSsgIh0b0grl8SFpQA9uz'
    ]},
    'жека' : {'канал' : 'UCk7l9hpgLbpfHJR3Ssz6Njw', 'листы': [
        'PLsK47AUy3CkLGW5hEhFo_IwimS1vrhNWz'
    ]},
    'дюмин' : {'канал' : 'UCHiDKtT9BLzCbb3sH_DWbDw', 'листы': [
        'PLbwZwB-W4n4OIO9VDT5cHOfaIHUx9OAZH',
        'OLAK5uy_lVwvyCdzL2jOoCDxFj0eAh7xL50FdTy9c'
    ]},
    'воровайки' : {'канал' : 'UCQ6S7-gdPqdnzP4jUV2gqjQ', 'листы': [
        'PLqSyQLbcuqeEWybyfRuLeWA0Uhj6z8cjO',
        'PLqSyQLbcuqeEcpfsW_2PYAQF_KyowTkWG'
    ]},
    'шуфутинский' : {'канал' : 'UC9G759R-e1jZ5xfsi6naqHw', 'листы': [
        'PLgcDoyCYSpOQ-P7G6WX-W6Lp78Zbi1hDm'
    ]}
}

In [25]:

for performer in target.keys():
    print('Качаю {}'.format(performer))
    
    channel_id = target[performer]['канал']
    playlist_ids =  target[performer]['листы']

    # видосы с основного канала 
    video_chanel = get_all_video_in_channel(channel_id, api_key)

    # видосы с плейлистов
    video_playlist = { }
    for lst in playlist_ids:
        video_playlist.update(get_all_video_in_playlist(lst, api_key))

    print('Видео с канала:',len(video_chanel))
    print('Видео с плейлиста:',len(video_playlist))
    video_playlist.update(video_chanel)
    print('Видео всего:',len(video_playlist))
    
    # пошли комменты
    def getCom(x):
        try:
            return get_comment_threads(x, api_key, style, performer, video_playlist)
        except:
            print('Error in https://www.youtube.com/watch?time_continue=1&v={}'.format(x))
            return [ ]

    n_jobs = -1 # параллелим на все ядра 
    result = Parallel(n_jobs=n_jobs)(delayed(getCom)(
        text) for text in tqdm_notebook(list(video_playlist.keys())))
    
    comments = [ ]
    for item in result:
        comments.extend(item)

    print('Добыто комментариев', len(comments))
    
    # воткнуть сюда сохранялку
    comments_dict[performer] = comments
    print("==================================================")

Качаю воровайки
Видео с канала: 90
Видео с плейлиста: 219
Видео всего: 291



Добыто комментариев 0


In [64]:
# Путь к данным
path = '/Users/fulyankin/Yandex.Disk.localized/Проекты/top_russian_music/comments/youtube/by_styles/'

with open(path + 'youtube_{}'.format(style), 'wb') as f:
    pickle.dump(comments_dict, f)

In [84]:
full_comments = [ ]
for item in comments_dict:
    full_comments.extend(comments_dict[item])
    
len(full_comments)

34795

In [63]:
full_comments[40000]

{'author': 'Василиса Красотина',
 'comment_id': 'Ugxsu5CqmaPr20Ocezl4AaABAg',
 'date': '2018-06-16T18:43:15.000Z',
 'likes': 0,
 'music_style': 'rock',
 'performer': 'ленинград',
 'text': '1:28 ФАРАОН?!?!?!',
 'video_name': 'Ленинград — Ч.П.Х.'}

In [70]:
comments_dict.keys()

dict_keys(['элизиум', 'план ломоносова', 'порнофильмы', 'йорш', 'король_и_шут'])